In [4]:
#Import libraries
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import metrics, svm, naive_bayes
from PIL import Image

In [2]:
#Function to extract labels from files
def extract_labels(file_path):
    with open(file_path) as file_labels:

        file_lines = file_labels.readlines()

        file_shape_labels= [None] * len(file_lines)
        file_position_labels = [None] * len(file_lines)
        file_phoneme_labels = [None] * len(file_lines)
        file_second_phoneme_label = [None] * len(file_lines)
        
        for i, line in enumerate(file_lines):
            file_line = line.split()

            file_shape_labels[i]  = int(file_line[1])
            file_position_labels[i] = int(file_line[2])
            file_phoneme_labels[i] = int(file_line[3])
            file_second_phoneme_label[i] = int(file_line[4])
            
    return np.array(file_shape_labels), np.array(file_position_labels), np.array(file_phoneme_labels), np.array(file_second_phoneme_label)

In [3]:
#Funtion to randomly shuffle data
def shuffle_data(images, shape_labels, position_labels, phoneme_labels, second_phoneme_labels):
    #index = np.arange(len(labels))
    #np.shuffle(index)
    index = np.random.choice(np.arange(len(phoneme_labels)), replace = False, size = len(phoneme_labels))
    
    shuffled_images = images[index]
    shuffled_shape_labels = shape_labels[index]
    shuffled_position_labels = position_labels[index]
    shuffled_phoneme_labels = phoneme_labels[index]
    shuffled_second_phoneme_labels = second_phoneme_labels[index]
    return shuffled_images, shuffled_shape_labels, shuffled_position_labels, shuffled_phoneme_labels, shuffled_second_phoneme_labels

In [4]:
#Function to split data
def split_data(data, split_ratio=0.9):
    partition = round(len(data) * split_ratio)
    
    train = data[:partition]
    test = data[partition:]
    return train, test

In [6]:
#Function to prepare dataset as grey images to input network
def create_grey_dataset(image_dir):
    grey_dataset = []
    for image_name in os.listdir(image_dir):
        image = Image.open(image_dir + image_name)
        image_as_array = np.asarray(image)
        r, g, b = image_as_array[:,:,0], image_as_array[:,:,1], image_as_array[:,:,2]
        grey_image = (0.3*r) + (0.59*g) +(0.11*b)
        grey_dataset.append(grey_image)
    return np.array(grey_dataset)

In [7]:
#Function to concatenate outputs of two networks into single vector
def join_outputs(out1, out2):
    joint_output = []
    for i in range(len(out1)):
        joint = np.concatenate((out1[i],out2[i]))
        joint_output.append(joint)
    return np.array(joint_output)

In [1]:
#Function to plot training and validation loss and accuracy
#Obtained from https://www.tensorflow.org/tutorials/images/classification
def plot_training_results(history, epochs = 5):
    
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.show()
    return

In [8]:
#Open image folders
consonants_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/ConsonantImages/')
vowels_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/VowelImages/')
combined_dir = os.path.join('/Users/User/Desktop/MSc Project/English Cued Speech/PhonemeImages/')

#Open label files
consonant_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Consonant.txt"
vowel_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_Vowel.txt"
combined_labels = "/Users/User/Desktop/MSc Project/English Cued Speech/Labels_All.txt"

In [9]:
#Extract labels
(vowel_shape_labels, vowel_position_labels, vowel_phoneme_labels, not_used) = extract_labels(vowel_labels)
(consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this) = extract_labels(consonant_labels)
(combined_shape_labels, combined_position_labels, combined_consonant_labels, combined_vowel_labels) = extract_labels(combined_labels)

In [43]:
#Grey Consonants Model

In [44]:
#Convert images of consonants to greyscale and prepare to input into network
grey_consonant_images = create_grey_dataset(consonants_dir)
grey_consonant_images = grey_consonant_images.reshape(len(grey_consonant_images), 256, 256, 1)

In [45]:
#Randomly shuffle consonant images and labels
grey_shuffled_consonants, grey_shuffled_consonant_shape_labels, grey_shuffled_consonant_position_labels, grey_shuffled_consonant_phoneme_labels, grey_nor_this = shuffle_data(grey_consonant_images, consonant_shape_labels, consonant_position_labels, consonant_phoneme_labels, or_this)

#Split training and test data
grey_consonant_train, grey_consonant_test = split_data(grey_shuffled_consonants)
grey_consonant_shape_labels_train, grey_consonant_shape_labels_test = split_data(grey_shuffled_consonant_shape_labels)
grey_consonant_position_labels_train, grey_consonant_position_labels_test = split_data(grey_shuffled_consonant_position_labels)
grey_consonant_phoneme_labels_train, grey_consonant_phoneme_labels_test = split_data(grey_shuffled_consonant_phoneme_labels)

In [83]:
#Build the network to recognise hand shape
grey_hand_shape_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(9, activation='softmax')
])

#Compile the networ
grey_hand_shape_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_hand_shape_model.fit(grey_consonant_train, grey_consonant_shape_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 936ms/step - loss: 1.9867 - accuracy: 0.2246
Epoch 2/5
43/43 [==============================] - 41s 948ms/step - loss: 1.7503 - accuracy: 0.3684
Epoch 3/5
43/43 [==============================] - 40s 933ms/step - loss: 1.4320 - accuracy: 0.5226
Epoch 4/5
43/43 [==============================] - 41s 949ms/step - loss: 1.1139 - accuracy: 0.6368
Epoch 5/5
43/43 [==============================] - 41s 955ms/step - loss: 0.8265 - accuracy: 0.7354


In [84]:
#Test the network and print it's performance
grey_hand_shape_test_loss, grey_hand_shape_test_accuracy = grey_hand_shape_model.evaluate(grey_consonant_test, grey_consonant_shape_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_hand_shape_test_loss, grey_hand_shape_test_accuracy*100))

5/5 [==============================] - 1s 271ms/step - loss: 1.0187 - accuracy: 0.6333
Test loss: 1.0186818838119507, Test accuracy: 63.333332538604736


In [48]:
#Build network to recognise consonant phoneme
grey_consonant_phoneme_model = tf.keras.models.Sequential([
  tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 1)),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Complie the network
grey_consonant_phoneme_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train the network
grey_consonant_phoneme_model.fit(grey_consonant_train, grey_consonant_phoneme_labels_train, epochs=5)

Epoch 1/5
43/43 [==============================] - 40s 928ms/step - loss: 2.9621 - accuracy: 0.1119
Epoch 2/5
43/43 [==============================] - 41s 942ms/step - loss: 2.6837 - accuracy: 0.2187
Epoch 3/5
43/43 [==============================] - 40s 924ms/step - loss: 2.4077 - accuracy: 0.3262
Epoch 4/5
43/43 [==============================] - 40s 940ms/step - loss: 2.0682 - accuracy: 0.4114
Epoch 5/5
43/43 [==============================] - 41s 946ms/step - loss: 1.7353 - accuracy: 0.4885


In [49]:
#Test the network and print it's performance
grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy = grey_consonant_phoneme_model.evaluate(grey_consonant_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_phoneme_test_loss, grey_consonant_phoneme_test_accuracy*100))

5/5 [==============================] - 1s 270ms/step - loss: 2.1323 - accuracy: 0.3667
Test loss: 2.132263660430908, Test accuracy: 36.666667461395264


In [50]:
#Obtain decisions from both networks and concatenate into a single decision vector
grey_hand_shape_decision = grey_hand_shape_model(grey_consonant_train)

grey_consonant_phoneme_decision = grey_consonant_phoneme_model(grey_consonant_train)

grey_consonant_decision = join_outputs(grey_hand_shape_decision, grey_consonant_phoneme_decision)

In [51]:
#Obtain test vectors to test network on
grey_hand_shape_decision_test = grey_hand_shape_model(grey_consonant_test)

grey_consonant_phoneme_decision_test = grey_consonant_phoneme_model(grey_consonant_test)

grey_consonant_decision_test = join_outputs(grey_hand_shape_decision_test, grey_consonant_phoneme_decision_test)

In [52]:
#Build network to make final decision of phoneme
grey_consonant_decision_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(25, activation='softmax')
])

#Compile network
grey_consonant_decision_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Train network
grey_consonant_decision_model.fit(grey_consonant_decision, grey_consonant_phoneme_labels_train, epochs=30)

Epoch 1/30
43/43 [==============================] - 0s 2ms/step - loss: 3.1596 - accuracy: 0.0726
Epoch 2/30
43/43 [==============================] - 0s 2ms/step - loss: 3.0519 - accuracy: 0.1534
Epoch 3/30
43/43 [==============================] - 0s 2ms/step - loss: 2.9128 - accuracy: 0.2380
Epoch 4/30
43/43 [==============================] - 0s 2ms/step - loss: 2.7441 - accuracy: 0.3269
Epoch 5/30
43/43 [==============================] - 0s 2ms/step - loss: 2.5669 - accuracy: 0.4047
Epoch 6/30
43/43 [==============================] - 0s 2ms/step - loss: 2.3940 - accuracy: 0.4211
Epoch 7/30
43/43 [==============================] - 0s 2ms/step - loss: 2.2329 - accuracy: 0.4344
Epoch 8/30
43/43 [==============================] - 0s 2ms/step - loss: 2.0910 - accuracy: 0.4255
Epoch 9/30
43/43 [==============================] - 0s 2ms/step - loss: 1.9708 - accuracy: 0.4618
Epoch 10/30
43/43 [==============================] - 0s 2ms/step - loss: 1.8716 - accuracy: 0.4685
Epoch 11/30
43/43 [

In [53]:
#Test network and print result
grey_consonant_test_loss, grey_consonant_test_accuracy = grey_consonant_decision_model.evaluate(grey_consonant_decision_test, grey_consonant_phoneme_labels_test)
print ('Test loss: {}, Test accuracy: {}'.format(grey_consonant_test_loss, grey_consonant_test_accuracy*100))

5/5 [==============================] - 0s 2ms/step - loss: 1.7051 - accuracy: 0.4200
Test loss: 1.70505952835083, Test accuracy: 41.999998688697815


In [110]:
#Build SVM classifier for consonant
consonant_svm = svm.SVC(kernel='linear')

consonant_svm.fit(grey_consonant_decision, grey_consonant_phoneme_labels_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [111]:
#Test SVM classifier and print accuracy
svm_consonant_decision = consonant_svm.predict(grey_consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(grey_consonant_phoneme_labels_test, svm_consonant_decision)*100)

In [63]:
#Build Naive-Bayes classifier for consonant
consonant_nb = naive_bayes.GaussianNB()

consonant_nb.fit(grey_consonant_decision, grey_consonant_phoneme_labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [66]:
#Test Naive-Bayes classifier and print accuracy
nb_consonant_decision = consonant_nb.predict(grey_consonant_decision_test)

print("Accuracy:",metrics.accuracy_score(grey_consonant_phoneme_labels_test, nb_consonant_decision)*100)